# Step 4: 다중 Tool 사용하기

이 노트북에서는 **여러 도구를 등록**하고 AI가 상황에 맞게 선택하도록 합니다.

**학습 목표:**
- 여러 도구 동시 등록
- AI의 자동 도구 선택
- 실용적인 챗봇 완성

---

## 1. 패키지 설치 및 모델 선택

In [1]:
!pip install anthropic -q
print("설치 완료!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 6.0 MB/s eta 0:00:00
설치 완료!


In [2]:
# ===== Claude 4.5 모델 선택 =====
# 아래 세 가지 모델 중 하나를 선택하세요 (주석 해제)

# 1. Haiku 4.5 - 가장 빠르고 저렴 (테스트용 추천)
# MODEL = "claude-haiku-4-5-20251001"

# 2. Sonnet 4.5 - 균형 잡힌 성능
MODEL = "claude-sonnet-4-5-20250929"

# 3. Opus 4.5 - 최고 성능 (비용 높음)
# MODEL = "claude-opus-4-5-20251101"

print(f"선택된 모델: {MODEL}")

선택된 모델: claude-sonnet-4-5-20250929


---

## 2. API 키 설정

In [3]:
import os
from datetime import datetime
import random
from anthropic import Anthropic

# API 키 설정
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-여기에-API-키-입력"
DEBUG = True
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
except:
    pass

client = Anthropic()
print("준비 완료!")

준비 완료!


---

## 3. 4가지 도구 정의

AI가 질문에 따라 적절한 도구를 선택합니다.

In [4]:
TOOLS = [
    {
        "name": "get_current_time",
        "description": "현재 날짜와 시간을 조회합니다.",
        "input_schema": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "calculator",
        "description": "두 숫자의 사칙연산을 수행합니다. operation: add, subtract, multiply, divide",
        "input_schema": {
            "type": "object",
            "properties": {
                "a": {"type": "number", "description": "첫 번째 숫자"},
                "b": {"type": "number", "description": "두 번째 숫자"},
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "연산 종류"
                }
            },
            "required": ["a", "b", "operation"]
        }
    },
    {
        "name": "get_weather",
        "description": "특정 도시의 현재 날씨(온도, 상태)를 조회합니다.",
        "input_schema": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "도시 이름 (예: 서울, 부산)"}
            },
            "required": ["city"]
        }
    },
    {
        "name": "random_number",
        "description": "지정된 범위(min~max)에서 랜덤 정수를 하나 생성합니다.",
        "input_schema": {
            "type": "object",
            "properties": {
                "min": {"type": "integer", "description": "최소값"},
                "max": {"type": "integer", "description": "최대값"}
            },
            "required": ["min", "max"]
        }
    }
]

print("=" * 60)
print("등록된 도구 목록")
print("=" * 60)
print(f"TOOLS 자료형: {type(TOOLS).__name__}")
print(f"TOOLS 길이: {len(TOOLS)}")
for i, tool in enumerate(TOOLS):
    print(f"  [{i}] {tool['name']}: {tool['description'][:30]}...")


등록된 도구 목록
TOOLS 자료형: list
TOOLS 길이: 4
  [0] get_current_time: 현재 날짜와 시간을 조회합니다....
  [1] calculator: 두 숫자의 사칙연산을 수행합니다. operation: ...
  [2] get_weather: 특정 도시의 현재 날씨(온도, 상태)를 조회합니다....
  [3] random_number: 지정된 범위(min~max)에서 랜덤 정수를 하나 생성...


---

## 4. 도구 실행 함수

In [5]:
def execute_tool(name: str, params: dict) -> str:
    """
    도구를 실행하고 결과를 문자열로 반환합니다.

    Args:
        name: 도구 이름 (str)
        params: 도구 파라미터 (dict)

    Returns:
        실행 결과 (str)
    """
    if DEBUG:
        print(f"\n  ⚙️  execute_tool 호출")
        print(f"     name: {name} (type: {type(name).__name__})")
        print(f"     params: {params} (type: {type(params).__name__})")

    if name == "get_current_time":
        now = datetime.now()
        result = now.strftime("%Y-%m-%d %H:%M:%S")

    elif name == "calculator":
        a = params["a"]
        b = params["b"]
        op = params["operation"]

        if op == "add":
            result = str(a + b)
        elif op == "subtract":
            result = str(a - b)
        elif op == "multiply":
            result = str(a * b)
        elif op == "divide":
            result = str(a / b) if b != 0 else "Error: 0으로 나눌 수 없음"
        else:
            result = f"Error: 알 수 없는 연산 {op}"

    elif name == "get_weather":
        city = params["city"]
        temp = random.randint(-5, 35)
        conditions = ["맑음", "흐림", "비", "눈", "안개"]
        condition = random.choice(conditions)
        result = f"{city}: {temp}°C, {condition}"

    elif name == "random_number":
        min_val = params["min"]
        max_val = params["max"]
        num = random.randint(min_val, max_val)
        result = str(num)

    else:
        result = f"Error: 알 수 없는 도구 {name}"

    if DEBUG:
        print(f"     result: {result} (type: {type(result).__name__})")

    return result



In [6]:

# ============================================================
# 4. 메인 챗봇 함수
# ============================================================

def chat(user_message: str, max_iterations: int = 10) -> str:
    """
    사용자 메시지를 받아 AI 응답을 반환합니다.
    병렬 호출과 연쇄 호출을 모두 지원합니다.

    Args:
        user_message: 사용자 질문 (str)
        max_iterations: 최대 반복 횟수 (int)

    Returns:
        AI 최종 응답 (str)
    """

    print("\n" + "=" * 60)
    print(f"📝 사용자 질문: {user_message}")
    print("=" * 60)

    # --------------------------------------------------------
    # Step 1: messages 초기화
    # --------------------------------------------------------
    # 자료형: list[dict]
    # 각 dict는 {"role": str, "content": str | list} 구조

    messages = [
        {
            "role": "user",           # str
            "content": user_message   # str
        }
    ]

    if DEBUG:
        print("\n[Step 1] messages 초기화")
        print(f"  messages 자료형: {type(messages).__name__}")
        print(f"  messages 길이: {len(messages)}")
        print(f"  messages[0]['role']: {messages[0]['role']} ({type(messages[0]['role']).__name__})")
        print(f"  messages[0]['content']: {messages[0]['content'][:50]}... ({type(messages[0]['content']).__name__})")

    # --------------------------------------------------------
    # Step 2: 반복 루프 시작
    # --------------------------------------------------------

    iteration = 0

    while iteration < max_iterations:
        iteration += 1
        print(f"\n{'─' * 60}")
        print(f"🔄 반복 {iteration}회차")
        print(f"{'─' * 60}")

        if DEBUG:
            print(f"\n[Step 2-{iteration}] API 호출 전 messages 상태")
            print(f"  messages 길이: {len(messages)}")
            for i, msg in enumerate(messages):
                content_preview = str(msg['content'])[:80] + "..." if len(str(msg['content'])) > 80 else str(msg['content'])
                print(f"  [{i}] role={msg['role']}, content={content_preview}")

        # --------------------------------------------------------
        # Step 3: API 호출
        # --------------------------------------------------------

        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            tools=TOOLS,          # list[dict]
            messages=messages     # list[dict]
        )

        # response 자료형: anthropic.types.message.Message

        if DEBUG:
            print(f"\n[Step 3] API 응답 분석")
            print(f"  response 자료형: {type(response).__name__}")
            print(f"  response.id: {response.id}")
            print(f"  response.stop_reason: {response.stop_reason}")
            print(f"  response.content 자료형: {type(response.content).__name__}")
            print(f"  response.content 길이: {len(response.content)}")

        # --------------------------------------------------------
        # Step 4: stop_reason 확인
        # --------------------------------------------------------

        if response.stop_reason == "tool_use":
            # AI가 도구 호출을 요청함
            print(f"\n  🔧 stop_reason='tool_use' → 도구 호출 필요")

            # --------------------------------------------------------
            # Step 5: 모든 tool_use 블록 처리
            # --------------------------------------------------------
            # response.content는 list[ToolUseBlock | TextBlock]
            # 병렬 호출 시 여러 ToolUseBlock이 있을 수 있음

            tool_results = []  # 자료형: list[dict]

            if DEBUG:
                print(f"\n[Step 5] response.content 순회")

            for block_idx, block in enumerate(response.content):
                # block 자료형: ToolUseBlock 또는 TextBlock

                if DEBUG:
                    print(f"\n  --- block[{block_idx}] ---")
                    print(f"  block 자료형: {type(block).__name__}")
                    print(f"  block.type: {block.type}")
                    print(f"  block.idx: {block_idx}/{len(response.content)}")

                if block.type == "tool_use":
                    # block 자료형: ToolUseBlock
                    # 속성: id (str), name (str), input (dict)

                    print(f"\n  🔧 Tool 호출 감지!")
                    print(f"     block.id: {block.id}")
                    print(f"     block.name: {block.name}")
                    print(f"     block.input: {block.input}")

                    if DEBUG:
                        print(f"     block.id 자료형: {type(block.id).__name__}")
                        print(f"     block.name 자료형: {type(block.name).__name__}")
                        print(f"     block.input 자료형: {type(block.input).__name__}")

                    # --------------------------------------------------------
                    # Step 6: 도구 실행
                    # --------------------------------------------------------

                    result = execute_tool(block.name, block.input)
                    # result 자료형: str

                    print(f"\n  ✅ 실행 결과: {result}")

                    # --------------------------------------------------------
                    # Step 7: tool_result dict 생성
                    # --------------------------------------------------------

                    tool_result_dict = {
                        "type": "tool_result",       # str (고정값)
                        "tool_use_id": block.id,     # str (block.id와 매칭!)
                        "content": result            # str (실행 결과)
                    }

                    if DEBUG:
                        print(f"\n[Step 7] tool_result_dict 생성")
                        print(f"  tool_result_dict: {tool_result_dict}")
                        print(f"  자료형: {type(tool_result_dict).__name__}")

                    tool_results.append(tool_result_dict)

                elif block.type == "text":
                    # AI가 텍스트도 함께 출력한 경우 (드물지만 가능)
                    if DEBUG:
                        print(f"  (텍스트 블록 발견: {block.text[:50]}...)")

            # --------------------------------------------------------
            # Step 8: tool_results 완성 확인
            # --------------------------------------------------------

            if DEBUG:
                print(f"\n[Step 8] tool_results 완성")
                print(f"  tool_results 자료형: {type(tool_results).__name__}")
                print(f"  tool_results 길이: {len(tool_results)}")
                for i, tr in enumerate(tool_results):
                    print(f"  [{i}] tool_use_id={tr['tool_use_id'][:20]}..., content={tr['content']}")

            # --------------------------------------------------------
            # Step 9: messages 업데이트
            # --------------------------------------------------------
            # 중요: assistant 메시지와 user 메시지를 순서대로 추가

            # 9-1) AI 응답 추가 (tool_use 블록들 포함)
            assistant_message = {
                "role": "assistant",
                "content": response.content  # list[ToolUseBlock | TextBlock]
            }
            messages.append(assistant_message)

            # 9-2) 도구 결과 추가 (모든 결과를 하나의 user 메시지에!)
            user_message_with_results = {
                "role": "user",
                "content": tool_results  # list[dict] ← 핵심!
            }
            messages.append(user_message_with_results)

            if DEBUG:
                print(f"\n[Step 9] messages 업데이트 완료")
                print(f"  messages 길이: {len(messages)}")
                print(f"  추가된 assistant message content 자료형: {type(assistant_message['content']).__name__}")
                print(f"  추가된 user message content 자료형: {type(user_message_with_results['content']).__name__}")

            # 다음 반복으로 계속
            continue

        # --------------------------------------------------------
        # Step 10: 최종 응답 (stop_reason == "end_turn")
        # --------------------------------------------------------

        elif response.stop_reason == "end_turn":
            print(f"\n  ✨ stop_reason='end_turn' → 최종 응답")

            for block in response.content:
                if block.type == "text":
                    final_answer = block.text

                    if DEBUG:
                        print(f"\n[Step 10] 최종 응답 추출")
                        print(f"  final_answer 자료형: {type(final_answer).__name__}")
                        print(f"  final_answer 길이: {len(final_answer)}")

                    print(f"\n{'=' * 60}")
                    print(f"💬 AI 최종 답변:")
                    print(f"{'=' * 60}")
                    print(final_answer)

                    return final_answer

        else:
            # 예상치 못한 stop_reason
            print(f"\n  ⚠️ 예상치 못한 stop_reason: {response.stop_reason}")
            break

    # 최대 반복 초과
    return "⚠️ 최대 반복 횟수 초과"

---

## 6. 다양한 질문 테스트

AI가 질문에 따라 다른 도구를 선택하는지 확인합니다.

In [ ]:
# 초기 유저가 보낸 메시지 + 툴 사용해야 합니다, 라고 한 응답 1개 + 툴 사용한 결과 1개

In [7]:
    # 테스트 1: 랜덤 숫자 2개 뽑아서 계산
    chat("1부터 50 사이 랜덤 숫자 2개를 뽑아서 더해줘")


📝 사용자 질문: 1부터 50 사이 랜덤 숫자 2개를 뽑아서 더해줘

[Step 1] messages 초기화
  messages 자료형: list
  messages 길이: 1
  messages[0]['role']: user (str)
  messages[0]['content']: 1부터 50 사이 랜덤 숫자 2개를 뽑아서 더해줘... (str)

────────────────────────────────────────────────────────────
🔄 반복 1회차
────────────────────────────────────────────────────────────

[Step 2-1] API 호출 전 messages 상태
  messages 길이: 1
  [0] role=user, content=1부터 50 사이 랜덤 숫자 2개를 뽑아서 더해줘

[Step 3] API 응답 분석
  response 자료형: Message
  response.id: msg_01V7wRevVYqPgkrZ1gYwrrue
  response.stop_reason: tool_use
  response.content 자료형: list
  response.content 길이: 2

  🔧 stop_reason='tool_use' → 도구 호출 필요

[Step 5] response.content 순회

  --- block[0] ---
  block 자료형: ToolUseBlock
  block.type: tool_use
  block.idx: 0/2

  🔧 Tool 호출 감지!
     block.id: toolu_01Npd8KuUAZbXdmbF3fPdztw
     block.name: random_number
     block.input: {'min': 1, 'max': 50}
     block.id 자료형: str
     block.name 자료형: str
     block.input 자료형: dict

  ⚙️  execute_tool 호출
     n

'1부터 50 사이에서 랜덤으로 뽑은 두 숫자는:\n- 첫 번째: **18**\n- 두 번째: **31**\n\n두 숫자를 더한 결과: **18 + 31 = 49**'

In [ ]:
    # 테스트 2: 날씨 기반 판단 + 계산
    chat("서울과 부산 날씨를 확인하고, 두 도시의 온도 차이를 계산해줘")


📝 사용자 질문: 서울과 부산 날씨를 확인하고, 두 도시의 온도 차이를 계산해줘

[Step 1] messages 초기화
  messages 자료형: list
  messages 길이: 1
  messages[0]['role']: user (str)
  messages[0]['content']: 서울과 부산 날씨를 확인하고, 두 도시의 온도 차이를 계산해줘... (str)

────────────────────────────────────────────────────────────
🔄 반복 1회차
────────────────────────────────────────────────────────────

[Step 2-1] API 호출 전 messages 상태
  messages 길이: 1
  [0] role=user, content=서울과 부산 날씨를 확인하고, 두 도시의 온도 차이를 계산해줘

[Step 3] API 응답 분석
  response 자료형: Message
  response.id: msg_01NQgvrsuxGC4YPk7VW9eVU9
  response.stop_reason: tool_use
  response.content 자료형: list
  response.content 길이: 3

  🔧 stop_reason='tool_use' → 도구 호출 필요

[Step 5] response.content 순회

  --- block[0] ---
  block 자료형: TextBlock
  block.type: text
  block.idx: 0/3
  (텍스트 블록 발견: 서울과 부산의 날씨를 확인하고 온도 차이를 계산해드리겠습니다....)

  --- block[1] ---
  block 자료형: ToolUseBlock
  block.type: tool_use
  block.idx: 1/3

  🔧 Tool 호출 감지!
     block.id: toolu_01Y7M5WWWgDjjphiusaVBeme
     block.name: get_w

'현재 날씨 정보입니다:\n\n- **서울**: 1°C, 눈 ❄️\n- **부산**: 6°C, 흐림 ☁️\n\n**온도 차이**: 5°C (부산이 서울보다 5도 더 따뜻합니다)'

user 지금 시간을 확인하고, 현재 '분' 값을 기준으로 1부터 그 숫자 사이의 랜덤 숫자를 뽑아줘

ai 먼저 시간을 확인하겠다, get_time과 인자

user [get_time과 인자의 결과값]

ai 현재 시간은 13시 9분입니다. 이제 1부터 9 사이의 랜덤 숫자를 뽑아드리겠습니다...,random nummberr(1,9)

user [random nummberr(1,9)]



In [ ]:
    # 테스트 3: 시간 + 랜덤
    chat("지금 시간을 확인하고, 현재 '분' 값을 기준으로 1부터 그 숫자 사이의 랜덤 숫자를 뽑아줘")


📝 사용자 질문: 지금 시간을 확인하고, 현재 '분' 값을 기준으로 1부터 그 숫자 사이의 랜덤 숫자를 뽑아줘

[Step 1] messages 초기화
  messages 자료형: list
  messages 길이: 1
  messages[0]['role']: user (str)
  messages[0]['content']: 지금 시간을 확인하고, 현재 '분' 값을 기준으로 1부터 그 숫자 사이의 랜덤 숫자를 뽑아... (str)

────────────────────────────────────────────────────────────
🔄 반복 1회차
────────────────────────────────────────────────────────────

[Step 2-1] API 호출 전 messages 상태
  messages 길이: 1
  [0] role=user, content=지금 시간을 확인하고, 현재 '분' 값을 기준으로 1부터 그 숫자 사이의 랜덤 숫자를 뽑아줘

[Step 3] API 응답 분석
  response 자료형: Message
  response.id: msg_01Ub8y3WgLmE7DbLsGmLJiNN
  response.stop_reason: tool_use
  response.content 자료형: list
  response.content 길이: 2

  🔧 stop_reason='tool_use' → 도구 호출 필요

[Step 5] response.content 순회

  --- block[0] ---
  block 자료형: TextBlock
  block.type: text
  block.idx: 0/2
  (텍스트 블록 발견: 먼저 현재 시간을 확인하고, 그 분 값을 기준으로 랜덤 숫자를 뽑아드리겠습니다....)

  --- block[1] ---
  block 자료형: ToolUseBlock
  block.type: tool_use
  block.idx: 1/2

  🔧 Tool 호출 감지!
     blo

'현재 시간은 **13시 9분**이고, 1부터 9 사이에서 뽑은 랜덤 숫자는 **7**입니다! 🎲'

In [ ]:
# 테스트 4: 복합 계산
chat("1~10 사이 숫자 3개를 뽑아서 모두 곱해줘")


📝 사용자 질문: 1~10 사이 숫자 3개를 뽑아서 모두 곱해줘

[Step 1] messages 초기화
  messages 자료형: list
  messages 길이: 1
  messages[0]['role']: user (str)
  messages[0]['content']: 1~10 사이 숫자 3개를 뽑아서 모두 곱해줘... (str)

────────────────────────────────────────────────────────────
🔄 반복 1회차
────────────────────────────────────────────────────────────

[Step 2-1] API 호출 전 messages 상태
  messages 길이: 1
  [0] role=user, content=1~10 사이 숫자 3개를 뽑아서 모두 곱해줘

[Step 3] API 응답 분석
  response 자료형: Message
  response.id: msg_01Vvz5HC2H5siXQYtvDMJyhj
  response.stop_reason: tool_use
  response.content 자료형: list
  response.content 길이: 3

  🔧 stop_reason='tool_use' → 도구 호출 필요

[Step 5] response.content 순회

  --- block[0] ---
  block 자료형: ToolUseBlock
  block.type: tool_use
  block.idx: 0/3

  🔧 Tool 호출 감지!
     block.id: toolu_01WS4L1BzqjtA7BPcidNcKy5
     block.name: random_number
     block.input: {'min': 1, 'max': 10}
     block.id 자료형: str
     block.name 자료형: str
     block.input 자료형: dict

  ⚙️  execute_tool 호출
     name: r

'1~10 사이에서 뽑은 3개의 숫자는:\n- 첫 번째: **3**\n- 두 번째: **8**\n- 세 번째: **3**\n\n이 숫자들을 모두 곱한 결과는 **3 × 8 × 3 = 72**입니다! 🎲'

---

## 7. AI의 도구 선택 원리

AI는 각 도구의 `description`을 읽고 질문에 가장 적합한 도구를 선택합니다.

```
질문: "서울 날씨 어때?"
        ↓
AI가 모든 도구의 description 분석:
  - get_current_time: "현재 날짜와 시간을 조회" → ✗
  - calculator: "사칙연산을 수행" → ✗
  - get_weather: "도시의 현재 날씨를 조회" → ✓ 일치!
  - random_number: "랜덤 숫자를 생성" → ✗
        ↓
get_weather 선택, {"city": "서울"} 전달
```

그래서 **description을 명확하게 작성**하는 것이 중요합니다!

---

## 8. 정리

### 이번 단계에서 배운 내용

| 개념 | 설명 |
|------|------|
| 다중 Tool 등록 | `tools=[TOOL1, TOOL2, ...]` |
| AI 자동 선택 | description 기반으로 적합한 도구 선택 |
| 도구 라우팅 | tool_name으로 실행 함수 분기 |

### 전체 학습 내용 요약

| 단계 | 주제 | 핵심 |
|------|------|------|
| Step 1 | API 기본 호출 | `messages.create()`, `stop_reason` |
| Step 2 | Tool 정의 | `name`, `description`, `input_schema` |
| Step 3 | Tool 실행 | Agentic Loop, `tool_result` |
| Step 4 | 다중 Tool | 여러 도구 등록, AI 자동 선택 |

### Claude 4.5 모델 비교

| 모델 | ID | 특징 |
|------|-----|------|
| Haiku 4.5 | claude-haiku-4-5-20251001 | 빠름, 저렴 |
| Sonnet 4.5 | claude-sonnet-4-5-20250929 | 균형 |
| Opus 4.5 | claude-opus-4-5-20251101 | 최고 성능 |

### Tool Calling 흐름 정리

```
1. 도구 정의 (JSON Schema)
        ↓
2. API 호출 시 tools 파라미터로 등록
        ↓
3. AI가 stop_reason="tool_use"로 도구 호출 요청
        ↓
4. 우리가 도구 실행
        ↓
5. tool_result로 결과 전달
        ↓
6. AI가 최종 응답 생성 (stop_reason="end_turn")
```

---

축하합니다! Tool Calling의 기본을 모두 배웠습니다.